# 데이콘 - 한국어 문장 관계 분류 경진대회

0. 데이터 로드

In [ ]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 32.8 MB/s 
     |████████████████████████████████| 6.5 MB 35.2 MB/s 
     |████████████████████████████████| 596 kB 38.1 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train = pd.read_csv('/content/train_data.csv')
test = pd.read_csv('/content/test_data.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [ ]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


1. 전처리

In [ ]:
train.isnull().sum()

index         0
premise       0
hypothesis    0
label         0
dtype: int64

In [ ]:
train['label'].value_counts()

entailment       8561
contradiction    8489
neutral          7948
Name: label, dtype: int64

In [ ]:
train['premise'].map(len).max()

90

In [ ]:
train['hypothesis'].map(len).max()

103

In [ ]:
max_seq_len = 100

In [ ]:
len(train)

24998

In [ ]:
valid = train[20000:]
train = train[:20000]

2. KoGPT2 model 0.54

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids = []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token
        sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']
        tokens = sent1_tokens + sent2_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids

In [ ]:
X_train = convert_examples_to_features(train['premise'], train['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 20000/20000 [00:15<00:00, 1252.92it/s]


In [ ]:
X_valid = convert_examples_to_features(valid['premise'], valid['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 4998/4998 [00:05<00:00, 895.37it/s]


In [ ]:
X_test = convert_examples_to_features(test['premise'], test['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 1666/1666 [00:02<00:00, 829.19it/s]


In [ ]:
train_label = train['label'].tolist()
valid_label = valid['label'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label)
y_valid = idx_encode.transform(valid_label)

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
print(label_idx)

{'contradiction': 0, 'entailment': 1, 'neutral': 2}


In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = 'accuracy')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=3, batch_size=32, validation_data=(X_valid, y_valid),
    callbacks = [early_stopping]
)

Epoch 1/3
625/625 [==============================] - 891s 1s/step - loss: 1.1759 - accuracy: 0.3688 - val_loss: 1.0366 - val_accuracy: 0.4296
Epoch 2/3
625/625 [==============================] - 874s 1s/step - loss: 0.9753 - accuracy: 0.5169 - val_loss: 0.8441 - val_accuracy: 0.6164
Epoch 3/3
625/625 [==============================] - 875s 1s/step - loss: 0.7882 - accuracy: 0.6468 - val_loss: 0.8454 - val_accuracy: 0.6180


In [ ]:
results = model.predict(X_test)

In [ ]:
preds = [np.argmax(val) for val in results]

In [ ]:
out = [list(label_idx.keys())[i] for i in preds]
out[:3]

['neutral', 'neutral', 'contradiction']

In [ ]:
results = pd.DataFrame({"index": test['index'], "label": out})

,index,label
0,0,neutral
1,1,neutral
2,2,contradiction
3,3,neutral
4,4,entailment


In [ ]:
results.to_csv("submission.csv", index=False)